In [ ]:
import pandas as pd
import os
from sys import path
path.append(r'C:\Users\fenklab\Desktop\super_bowl_screen-main\Software')
from bowl_stimulate_class import *

stim = Stimulation_Pipeline()
log = stim.LoopScenes(arena = stim,
                      bouncing_limits = 60, side_duration = 3.0, side_per_scene = 4, break_duration =1.0,
                      framerate = 60, iteration = 20, inverted=True, rot_offset=(0,60,0), random_order = False,
                      scenes = [
                          [
                              "Black Bar on White",
                              stim.bar_vertical(width=30, color=0, color_b=125),
                              None,
                              lambda f, b, s : (stim.superpose(foreground = f, background = b, shift = s))
                              ],
                          [
                              "Checker Bar on Cheker",
                              stim.checker_bar_vertical(square_size=15, width_in_squares=2, color1=0, color2=80, color_b=-1),
                              stim.checker_screen(pixel_size=15, color1=0, color2=80),
                              lambda f, b, s : (stim.superpose(foreground = f, background = b, shift = s))
                              ],
                          [
                              "Black Bar on Checker",
                              stim.bar_vertical(width=30, color=0, color_b=-1),
                              stim.checker_screen(pixel_size=15, color1=0, color2=80),
                              lambda f, b, s : (stim.superpose(foreground = f, background = b, shift = s))
                              ],
                          [ # convergent and divergent scenes can be done by a background mirroring the foreground
                              "Checker Screen", #name of the scene
                              stim.checker_screen(pixel_size=15, color1=0, color2=80),
                              None,
                              lambda f, b, s : (stim.superpose(foreground = f, background = b, shift = s))
                              ]
                          ])


if len(sys.argv) > 1:
    output_directory_path = sys.argv[1]
    flyID = sys.argv[2]
    log.to_csv(os.path.join(output_directory_path, flyID + "_" + 'stimulus_log.csv'), index=False)

cv2.destroyAllWindows()
key = cv2.waitKey(1)